In [2]:
%pylab inline
import pandas as pd
from sklearn import metrics

Populating the interactive namespace from numpy and matplotlib


In [6]:
# Read in the prediction scores on the test set accross all three models
test_multiple_models = pd.read_csv('../data/test_score_multiple_models.csv')
test_multiple_models.head()

,pred_rf,pred_gb,pred_lr,label
0,0.157483,0.050373,0.012200,False
1,0.007528,0.017296,0.021629,False
2,0.010931,0.019073,0.013231,False
3,0.014331,0.026936,0.109715,False
4,0.002645,0.017125,0.005357,False


### Permutation Test
- randomly select with replacement from the test set
- Compute the auc for each model
- Compute the proportion of times model A > model B: $P(AUC_A > AUC_B)$

In [7]:
np.random.seed(12345)
N = test_multiple_models.shape[0]
bootstraps = 1000
res_list = []
for b in range(bootstraps):
    idx = np.random.choice(range(N), N, replace=True)
    res = {}
    for model in ['rf', 'gb', 'lr']:
        tmp = test_multiple_models.iloc[idx]
        probas = tmp['pred_'+model]
        y = tmp['label']
        fpr, tpr, thresholds = metrics.roc_curve(y, probas)
        try:
            auc = metrics.auc(fpr, tpr)
        except:
            auc = np.nan
        res.update({model: auc})
    res_list.append(res)
auc_bootstrapped = pd.DataFrame(res_list)

In [8]:
auc_bootstrapped.mean()

rf    0.875384
gb    0.867455
lr    0.855910
dtype: float64

$P(AUC_{linear regression} > AUC_{random forest})$

In [9]:
(auc_bootstrapped['lr'] > auc_bootstrapped['rf']).mean()

0.012

$P(AUC_{linear regression} > AUC_{gradient boosting})$

In [10]:
(auc_bootstrapped['lr'] > auc_bootstrapped['gb']).mean()

0.109

$P(AUC_{gradient boosting} > AUC_{random forest})$

In [11]:
(auc_bootstrapped['gb'] > auc_bootstrapped['rf']).mean()

0.015